<a href="https://colab.research.google.com/github/Jasmeetk05/2nd-year-project/blob/main/Copy_of_Untitled59.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch sentencepiece scikit-learn numpy tqdm flask


In [ ]:
!ngrok config add-authtoken "323g1mtR9moA9F2y4FClQhTI2dz_5PtgpJqGKJ5nthEf5qKf6"


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


### For our app interface
Step 1. Install dependencies

In [ ]:
# Cell 1 — install dependencies (run first)
!pip install -q streamlit pyngrok bcrypt transformers torch sentencepiece scikit-learn fpdf textstat spacy textblob matplotlib nltk
# Optional: if you want language_tool (Java) uncomment below (not required)
# !pip install -q language_tool_python

# Download spaCy and NLTK models
!python -m spacy download en_core_web_sm >/dev/null 2>&1 || true

import nltk
nltk.download('punkt')
print("✅ Install step finished.")



✅ Install step finished.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# 🧠 Cell 1 — setup dependencies and corpora
!pip install textblob language_tool_python pyngrok streamlit reportlab
!python -m textblob.download_corpora


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.6/55.6 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 23.7 MB/s eta 0:00:00
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Unzipping corpora/conll2000.zip.
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.
Finished.


Step 2: Create app Structure

In [ ]:
# Cell 2 — write app.py (run this second)
app_code = """
import streamlit as st
import sqlite3, bcrypt, random, time
from datetime import datetime
import matplotlib.pyplot as plt
from fpdf import FPDF
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import textstat
import spacy
from transformers import pipeline
from textblob import TextBlob
import difflib
import os

# ---------- CONFIG ----------
DB = "users.db"
FEEDBACK_MODEL = "google/flan-t5-small"   # instruction-following small model for feedback & rewriting
DEVICE = 0 if __import__('torch').cuda.is_available() else -1

# ---------- DATABASE ----------
conn = sqlite3.connect(DB, check_same_thread=False)
c = conn.cursor()
c.execute('''CREATE TABLE IF NOT EXISTS users (
    email TEXT PRIMARY KEY,
    password BLOB NOT NULL
)''')
c.execute('''CREATE TABLE IF NOT EXISTS history (
    email TEXT,
    essay TEXT,
    score REAL,
    feedback TEXT,
    grammar_score REAL,
    coherence_score REAL,
    plagiarism_score REAL,
    date TEXT
)''')
conn.commit()

# Auto-migration (add missing columns if schema older)
existing = [row[1] for row in c.execute("PRAGMA table_info(history)").fetchall()]
for col, ctype in [("feedback","TEXT"), ("grammar_score","REAL"), ("coherence_score","REAL"), ("plagiarism_score","REAL")]:
    if col not in existing:
        try:
            c.execute(f"ALTER TABLE history ADD COLUMN {col} {ctype}")
            conn.commit()
        except Exception:
            pass

# ---------- Session state init ----------
if "logged_in" not in st.session_state:
    st.session_state["logged_in"] = False
    st.session_state["user_email"] = None
if "otp_storage" not in st.session_state:
    st.session_state["otp_storage"] = {}
if "feedback_model" not in st.session_state:
    st.session_state["feedback_model"] = None
if "rewriter" not in st.session_state:
    st.session_state["rewriter"] = None
if "spacy_nlp" not in st.session_state:
    try:
        st.session_state["spacy_nlp"] = spacy.load("en_core_web_sm")
    except:
        st.session_state["spacy_nlp"] = None

# ---------- AUTH ----------
def signup(email, password):
    c.execute("SELECT * FROM users WHERE email=?", (email,))
    if c.fetchone(): return False
    hashed = bcrypt.hashpw(password.encode(), bcrypt.gensalt())
    c.execute("INSERT INTO users (email, password) VALUES (?,?)", (email, hashed))
    conn.commit(); return True

def login(email, password):
    c.execute("SELECT password FROM users WHERE email=?", (email,))
    row = c.fetchone()
    return bool(row and bcrypt.checkpw(password.encode(), row[0]))

def send_otp(email):
    otp = random.randint(100000,999999)
    st.session_state["otp_storage"][email] = otp
    st.info(f"(Demo OTP) sent to {email}: {otp}")

def verify_otp(email, entered):
    try:
        return st.session_state["otp_storage"].get(email) == int(entered)
    except:
        return False

# ---------- HISTORY DB ----------
def save_history(email, essay, score, feedback, g, co, pl):
    c.execute("INSERT INTO history (email, essay, score, feedback, grammar_score, coherence_score, plagiarism_score, date) VALUES (?,?,?,?,?,?,?,?)",
              (email, essay, score, feedback, g, co, pl, datetime.now().strftime("%Y-%m-%d %H:%M:%S")))
    conn.commit()

def get_history(email):
    c.execute("SELECT essay, score, feedback, grammar_score, coherence_score, plagiarism_score, date FROM history WHERE email=? ORDER BY date DESC", (email,))
    return c.fetchall()

# ---------- METRICS ----------
def grammar_score(essay):
    # TextBlob-based simple heuristic (corrections count)
    tb = TextBlob(essay)
    corrections = 0
    for s in tb.sentences:
        corrected = str(s.correct())
        if corrected.strip().lower() != str(s).strip().lower():
            corrections += 1
    score = max(0, 10 - corrections)
    return round(min(score, 10),2)

def coherence_score(essay):
    nlp = st.session_state.get("spacy_nlp")
    if nlp:
        sents = [s.text.strip() for s in nlp(essay).sents if len(s.text.strip())>10]
    else:
        sents = [s.strip() for s in essay.split('.') if len(s.strip())>10]
    if len(sents) < 2:
        return 5.0
    try:
        vec = TfidfVectorizer().fit_transform(sents)
        sims = cosine_similarity(vec)
        n = sims.shape[0]
        sim_score = (sims.sum() - sims.trace())/(n*(n-1)) if n>1 else 0.0
    except:
        sim_score = 0.2
    # transition words
    transitions = ["however","moreover","therefore","furthermore","consequently","nevertheless","in addition"]
    tcount = sum(essay.lower().count(t) for t in transitions)
    s_trans = min(tcount/3,1.0)
    s_sim = min(sim_score*2.0,1.0)
    score = round((s_trans*0.4 + s_sim*0.6)*10,2)
    return score

def plagiarism_score(essay, email):
    past = get_history(email)
    if not past:
        return 0.0
    past_texts = [p[0] for p in past]
    try:
        vec = TfidfVectorizer().fit_transform(past_texts + [essay])
        sims = cosine_similarity(vec[-1], vec[:-1]).flatten()
        max_sim = float(sims.max()) if len(sims)>0 else 0.0
    except:
        max_sim = 0.0
    return round(max_sim*100,2)

def overall_score(essay,email):
    g = grammar_score(essay)
    co = coherence_score(essay)
    pl = plagiarism_score(essay,email)
    length_factor = min(len(essay)/800,1.0)*10
    base = (g*0.30 + co*0.30 + length_factor*0.20)
    penalty = (pl/100)*10*0.20
    score = max(0.0, round(base - penalty,2))
    return score, g, co, pl

# ---------- AI FEEDBACK & REWRITER ----------
@st.cache_resource
def load_feedback_model():
    return pipeline("text2text-generation", model=FEEDBACK_MODEL, device=DEVICE)

def generate_feedback(essay):
    model = st.session_state.get("feedback_model") or load_feedback_model()
    st.session_state["feedback_model"] = model
    prompt = ("You are an expert academic writing tutor. Provide 4 concise, actionable bullets to improve the essay: "
              "1) Structure, 2) Grammar/style, 3) Vocabulary, 4) Concrete next steps. Keep bullets short.\\n\\nEssay:\\n" + essay[:1600])
    out = model(prompt, max_length=200, do_sample=False)
    text = out[0].get("generated_text", out[0].get("text","")).strip()
    if text.startswith(prompt): text = text[len(prompt):].strip()
    return text

@st.cache_resource
def load_rewriter():
    return pipeline("text2text-generation", model=FEEDBACK_MODEL, device=DEVICE)

def rewrite_professional(essay):
    rewriter = st.session_state.get("rewriter") or load_rewriter()
    st.session_state["rewriter"] = rewriter
    prompt = "Rewrite the following essay to be more professional, concise, and academic while preserving meaning:\\n\\n" + essay[:1600]
    out = rewriter(prompt, max_length=600, do_sample=False)
    text = out[0].get("generated_text", out[0].get("text","")).strip()
    if text.startswith(prompt): text = text[len(prompt):].strip()
    return text

# ---------- Grammar correction highlight ----------
def corrected_text(essay):
    tb = TextBlob(essay)
    corrected = str(tb.correct())
    return corrected

def highlight_differences(original, corrected):
    # token-level simple highlight: wrap corrected tokens that differ
    orig_tokens = original.split()
    corr_tokens = corrected.split()
    seqm = difflib.SequenceMatcher(a=orig_tokens, b=corr_tokens)
    out = []
    for op, a0, a1, b0, b1 in seqm.get_opcodes():
        if op == "equal":
            out.extend(orig_tokens[a0:a1])
        elif op in ("replace", "insert"):
            # show corrected tokens wrapped
            out.append("<mark>" + " ".join(corr_tokens[b0:b1]) + "</mark>")
        elif op == "delete":
            # show deletion as strike
            out.append("<del>" + " ".join(orig_tokens[a0:a1]) + "</del>")
    return " ".join(out)

# ---------- PDF report ----------
def make_pdf(email, essay, score, feedback, g, co, pl, rewritten=None):
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=12)
    pdf.cell(0,10, "NLP Essay Grader Report", ln=True, align="C")
    pdf.ln(4)
    pdf.cell(0,8, f"User: {email}", ln=True)
    pdf.cell(0,8, f"Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}", ln=True)
    pdf.cell(0,8, f"Overall Score: {score}/10", ln=True)
    pdf.cell(0,8, f"Grammar: {g}/10  |  Coherence: {co}/10  |  Plagiarism: {pl}%", ln=True)
    pdf.ln(6)
    pdf.multi_cell(0,6,"Feedback:\\n" + feedback)
    if rewritten:
        pdf.ln(4)
        pdf.multi_cell(0,6,"Rewritten (professional):\\n" + rewritten[:4000])
    else:
        pdf.ln(4)
        pdf.multi_cell(0,6,"Essay:\\n" + essay[:4000])
    out = f"report_{int(time.time())}.pdf"
    pdf.output(out)
    return out

# ---------- STREAMLIT UI ----------
st.set_page_config(page_title="Professional AI Essay Grader", layout="wide")
st.markdown(\"\"\"<style>
body { background: linear-gradient(180deg,#f7f9fb,#eef2f7); }
.card { background:white; border-radius:12px; padding:16px; box-shadow:0 6px 18px rgba(0,0,0,0.06); }
.stButton>button { background-color:#0b63ce; color:white; border-radius:8px; height:42px; }
div[data-testid='stSidebar'] { background-color:#f2f4f8; }
mark { background-color: #fffb91; }
del { color:#b0b0b0; text-decoration: line-through; }
</style>\"\"\", unsafe_allow_html=True)

# Header
col1, col2 = st.columns([3,1])
with col1:
    st.title("NLP Essay Grader — Professional Dashboard")
    st.caption("AI feedback, grammar corrections, rewriter, coherence & plagiarism checks, downloadable reports.")
with col2:
    st.image("https://cdn-icons-png.flaticon.com/512/2917/2917242.png", width=64)

# Auth
if not st.session_state["logged_in"]:
    left, right = st.columns(2)
    with left:
        st.subheader("Login")
        email = st.text_input("Email", key="login_e")
        password = st.text_input("Password", type="password", key="login_p")
        if st.button("Login", key="btn_login"):
            if login(email, password):
                st.session_state["logged_in"] = True
                st.session_state["user_email"] = email
                st.success("Logged in")
                st.experimental_rerun()
            else:
                st.error("Invalid credentials")
    with right:
        st.subheader("Sign Up")
        new_email = st.text_input("New Email", key="signup_e")
        new_password = st.text_input("New Password", type="password", key="signup_p")
        if st.button("Send OTP", key="btn_otp"):
            if new_email and new_password:
                send_otp(new_email)
            else:
                st.warning("Enter email & password")
        otp_in = st.text_input("Enter OTP", key="otp_in")
        if st.button("Verify & Create Account", key="btn_create"):
            if verify_otp(new_email, otp_in):
                if signup(new_email, new_password):
                    st.success("Account created — please log in")
                    st.session_state["otp_storage"].pop(new_email, None)
                else:
                    st.warning("Email exists")
            else:
                st.error("Invalid OTP")
else:
    st.sidebar.success(f"Signed in as {st.session_state['user_email']}")
    if st.sidebar.button("Logout"):
        st.session_state["logged_in"] = False
        st.session_state["user_email"] = None
        st.experimental_rerun()

    page = st.sidebar.radio("Navigate", ["Grader", "History", "Account", "Rewrite & Correct"])

    if page == "Grader":
        st.subheader("Write or paste your essay")
        essay = st.text_area("Essay", height=300)
        if st.button("Evaluate"):
            if not essay.strip():
                st.warning("Please enter text")
            else:
                with st.spinner("Computing feedback and metrics..."):
                    feedback = generate_feedback(essay)
                    score, g, co, pl = overall_score(essay, st.session_state["user_email"])
                    save_history(st.session_state["user_email"], essay, score, feedback, g, co, pl)
                    corrected = corrected_text(essay)
                    highlighted = highlight_differences(essay, corrected)
                st.success(f"Overall score: {score}/10")
                st.markdown(\"\"\"<div class='card'><h4>AI Feedback</h4><div>\"\"\" + feedback.replace('\\n','<br>') + \"\"\"</div></div>\"\"\", unsafe_allow_html=True)
                st.markdown("### Grammar correction (suggested)")
                st.markdown(highlighted, unsafe_allow_html=True)
                st.markdown("### Metrics")
                c1,c2,c3 = st.columns(3)
                c1.metric("Grammar", f\"{g}/10\")
                c2.metric("Coherence", f\"{co}/10\")
                c3.metric("Plagiarism", f\"{pl}%\")
                # PDF
                pdf_path = make_pdf(st.session_state["user_email"], essay, score, feedback, g, co, pl, rewritten=None)
                with open(pdf_path,"rb") as f:
                    st.download_button("Download Report (PDF)", f, file_name=pdf_path)
                st.balloons()

    elif page == "Rewrite & Correct":
        st.subheader("Rewrite to Professional & Grammar Correction")
        essay2 = st.text_area("Paste essay to rewrite or correct", height=300)
        colA, colB = st.columns(2)
        with colA:
            if st.button("Show Grammar Correction"):
                if not essay2.strip():
                    st.warning("Enter essay")
                else:
                    corr = corrected_text(essay2)
                    highlighted = highlight_differences(essay2, corr)
                    st.markdown("### Corrected (highlighted differences):")
                    st.markdown(highlighted, unsafe_allow_html=True)
        with colB:
            if st.button("Rewrite Professionally"):
                if not essay2.strip():
                    st.warning("Enter essay")
                else:
                    with st.spinner("Generating professional rewrite..."):
                        rewritten = rewrite_professional(essay2)
                    st.markdown("### Professional Rewrite")
                    st.write(rewritten)
                    # allow download
                    temp_pdf = make_pdf(st.session_state["user_email"] or "guest", essay2, 0, rewritten, 0,0,0, rewritten=rewritten)
                    with open(temp_pdf,"rb") as f:
                        st.download_button("Download Rewritten (PDF)", f, file_name=temp_pdf)

    elif page == "History":
        st.subheader("Your History")
        history = get_history(st.session_state["user_email"])
        if not history:
            st.info("No history")
        else:
            scores = [h[1] for h in history][::-1]
            dates = [h[6] for h in history][::-1]
            fig, ax = plt.subplots(figsize=(8,3))
            ax.plot(dates, scores, marker='o')
            ax.set_xlabel("Date"); ax.set_ylabel("Score")
            plt.xticks(rotation=25)
            st.pyplot(fig)
            for h in history:
                st.markdown(f\"**Date:** {h[6]}  |  **Score:** {h[1]}/10  |  **Plagiarism:** {h[5]}%\")
                st.info(h[2])
                st.write(h[0][:400] + ("..." if len(h[0])>400 else ""))

    else:
        st.subheader("Account")
        st.write(f\"Email: **{st.session_state['user_email']}**\")
        st.write(\"This app includes: AI feedback, grammar highlight, professional rewriter, downloadable reports.\")
"""

with open("app.py","w") as f:
    f.write(app_code)

print("✅ app.py written.")



✅ app.py written.


In [ ]:
# Cell 3 — run app and expose via ngrok
from pyngrok import ngrok
import time, os

# kill old
!pkill -f ngrok || true
!pkill -f streamlit || true
time.sleep(1)

# run streamlit in background
get_ipython().system_raw('streamlit run app.py --server.port 8501 &')

time.sleep(5)

# disconnect previous tunnels (best effort)
try:
    for t in ngrok.get_tunnels():
        try:
            ngrok.disconnect(t.public_url)
        except:
            pass
except Exception:
    pass

try:
    tunnel = ngrok.connect(addr=8501, proto="http")
    print("🚀 App URL:", tunnel.public_url)
except Exception as e:
    print("Could not start ngrok tunnel:", e)
    print("If you see a tunnel-limit error, restart the Colab runtime (Runtime -> Restart runtime) and re-run the three cells.")


^C
^C
🚀 App URL: https://c39e05577dc2.ngrok-free.app
